# 🧼 Clean & Combine CPI Data for Tableau
**Goal**: Load multiple BLS CPI Excel files, clean them, reshape them, and combine into a single dataset ready for Tableau.

Each sheet should come from BLS regional data for CPI categories like Food, Shelter, Energy, etc.
This notebook also calculates Year-over-Year (YoY) percent changes.

In [12]:
import pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

## 🔧 Function to Clean Individual Excel File

In [13]:
def load_and_clean_cpi_excel(file_path, category_name):
    xls = pd.ExcelFile(file_path)
    df_raw = xls.parse(xls.sheet_names[0], skiprows=11)

    df_long = df_raw.melt(id_vars='Year', value_vars=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                                                      'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
                          var_name='Month', value_name='CPI')
    df_long = df_long.dropna(subset=['CPI'])

    month_lookup = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
                    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
    df_long['Month_Num'] = df_long['Month'].map(month_lookup)
    df_long['Date'] = pd.to_datetime(dict(year=df_long['Year'], month=df_long['Month_Num'], day=1))
    df_long['Category'] = category_name
    return df_long[['Date', 'Category', 'CPI']].sort_values('Date')

## 🧩 Function to Combine All Categories

In [14]:
def combine_categories(file_category_map):
    combined_df = pd.DataFrame()
    for file_path, category in file_category_map.items():
        df = load_and_clean_cpi_excel(file_path, category)
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    return combined_df

## 📈 Function to Calculate YoY % Change

In [15]:
def calculate_yoy_change(df):
    df = df.copy()
    df['YoY_Change'] = df.groupby('Category')['CPI'].pct_change(periods=12) * 100
    return df

## 💾 Save Final Data for Tableau

In [16]:
def save_for_tableau(df, output_path):
    df.to_csv(output_path, index=False)
    print(f"✅ Saved cleaned data to: {output_path}")

## 🚀 Run All Functions to Generate Final Dataset

In [17]:
# Replace with your actual file paths
file_category_map = {
    'data/food.xlsx': 'Food',
    'data/energy.xlsx': 'Energy',
    'data/shelter.xlsx': 'Shelter',
    'data/transportation.xlsx': 'Transportation',
    'data/all_items.xlsx': 'All_Items'
}

df_combined = combine_categories(file_category_map)
df_final = calculate_yoy_change(df_combined)
save_for_tableau(df_final, 'virginia_cpi_tableau_ready.csv')

✅ Saved cleaned data to: virginia_cpi_tableau_ready.csv


In [10]:
!ls data/

All_Items.xlsx      Energy.xlsx         Houseing.xlsx       Transportation.xlsx
energy_old.xlsx     Food.xlsx           Shelter.xlsx


In [9]:
pwd

'/Users/aravindanisetti/Desktop/2025 Training/Data Roadmap'